In [ ]:
import os
import datetime as dt
from datetime import datetime, date
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
os.chdir('C:\\Users\\Romina\\Desktop\\thesis')
os.getcwd()

In [ ]:
#df= pd.read_csv("log_data_final.csv", low_memory=False)

df= pd.read_csv("final_session_30mins.csv", low_memory=False)


In [ ]:
df.head(100)

In [ ]:
list(df.columns)

In [ ]:
#df.head(30)
#df.info()

In [ ]:
#df.info()
#df['_source.doc.@fields.event'].unique()

In [ ]:
#identify sessions with queries- that is sessions with a query term search which include the sessions with atleast one event
grouped = df.groupby('session_no')['ip','q'].count().reset_index()
grouped.head()
#non_abandon=grouped[(grouped['_source.user.ip'] > 1) & (grouped['_source.doc.@fields.filters.q'] > 0)]
#SESSIONS With atleast a search Query- this includes both abandoned and non-abandoned queries
sessionsWithQ=grouped[(grouped['q'] > 0)]
sessionsWithQ.head()

In [ ]:
#check the sessions with one event that have a query and they are abandons in our case (same as sessions without click)
# test=non_abandon[non_abandon['_source.user.ip'] == 1]
# test.head()

In [ ]:
sessionsWithQ_dat=df[df['session_no'].isin(sessionsWithQ['session_no'])].reset_index()
sessionsWithQ_dat=sessionsWithQ_dat.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
sessionsWithQ_dat.head()
#non_abandon_dat.info()

In [ ]:
# get the result id for all the shown results
sessions=sessionsWithQ_dat[['session_no', 'ip','event','http_referer','url',
                          'result_id','q']]
sessions.head(100)
#sessions.to_csv("sessions.csv")

In [ ]:
sessions.info()

In [ ]:
#flag sessions with views (click) and those accessed portal (Go To Data Provider, Go To Service Provider))
grouped_event = sessions.groupby(['session_no','event'])['ip'].count().reset_index()
grouped_event.head()

#pivot and append to the entire data
grouped_event_=grouped_event.pivot(index='session_no', columns='event', values='ip')
grouped_event_=grouped_event_[['portal_search','portal_view', 'portal_accessed', 'portal_export']]
grouped_event_.head()

In [ ]:
#append the views and accessed back to the data
sessions=pd.merge(sessions, grouped_event_, left_on='session_no', right_on='session_no', how='left')
sessions.info()
sessions.head()

In [ ]:
sessions_=pd.DataFrame(sessions.result_id.str.split(',', expand=True).values).add_prefix('Result_id_')

In [ ]:
results_top_5=sessions_.iloc[ :, 0:5]
results_top_5.head(10)

In [ ]:
# keeping the top 5 for the timebeing
results_top_5_= results_top_5.apply(lambda x: x.str.strip("[]"))
results_top_5_= results_top_5_.apply(lambda x: x.str.replace(r"'", '' ))
results_top_5_= results_top_5_.apply(lambda x: x.str.strip())
results_top_5_.head(30)

In [ ]:
import numpy as np
results_top_5_=results_top_5_.replace(r'', np.NaN)
results_top_5_.fillna(value=pd.np.nan, inplace=True)
#results_top_5_=results_top_5_.replace(r'None', np.NaN)
results_top_5_.head(30)
#results_top_5_=results_top_5_.dropna(subset=['Result_id_0'])

In [ ]:
#append back to the entire data
sessions.reset_index(drop=True, inplace=True)
results_top_5_.reset_index(drop=True, inplace=True)
non_abandon_sessions=pd.concat([sessions, results_top_5_], axis=1, sort=False)
non_abandon_sessions.head()

In [ ]:
# there are sessions that have no query but results- removing them for the moment- we can revisit if required
non_abandon_sessions_= non_abandon_sessions[non_abandon_sessions['q'].notnull()]

non_abandon_sessions_.info()
non_abandon_sessions_.head()

In [ ]:
#subjects with anz codes
titles_anz=pd.read_csv('subject_data.csv')
titles_anz.head()
# titles['id']=titles['id'].astype(str)
titles_anz.info()
titles_anz.head()
# # subjects with tsubject
# titles_t=pd.read_csv('titles_tsub_id.csv')
# titles_t.head()
# titles.info()
titles_anz.drop(columns=['Unnamed: 0'], inplace=True)
titles_anz=titles_anz.astype(str)


In [ ]:
# join the titles to the relvant Id for all the top 5 resutls shown
data_1=pd.merge(non_abandon_sessions_, titles_anz[['title','id','final_sub0','final_sub1','final_sub2',
                                               'final_sub3', 'final_sub4','final_sub5']],
                                                left_on='Result_id_0', right_on='id', how='left'); data_1=data_1.drop(columns=['id']); data_1=data_1.rename(columns={"title": "title_id_0"})
data_2=pd.merge(data_1, titles_anz[['id', 'title']], left_on='Result_id_1', right_on='id', how='left'); data_2=data_2.drop(columns=['id']); data_2=data_2.rename(columns={"title": "title_id_1"})
data_3=pd.merge(data_2, titles_anz[['id', 'title']], left_on='Result_id_2', right_on='id', how='left'); data_3=data_3.drop(columns=['id']); data_3=data_3.rename(columns={"title": "title_id_2"})
data_4=pd.merge(data_3,titles_anz[['id', 'title']], left_on='Result_id_3', right_on='id', how='left'); data_4=data_4.drop(columns=['id']); data_4=data_4.rename(columns={"title": "title_id_3"})
data_final=pd.merge(data_4, titles_anz[['id', 'title']], left_on='Result_id_4', right_on='id', how='left'); data_final=data_final.drop(columns=['id']); data_final=data_final.rename(columns={"title": "title_id_4"})
data_final.info()

In [ ]:
data_final.head(50)


In [ ]:
test=data_final[data_final['Result_id_0'].isnull()]
test.info()

In [ ]:
#data_final.to_csv("final_matched_data_v2.csv")
data_final.to_csv("final_matched_data_session30.csv")